In [1]:
import pftoolbox
import pandas as pd
import numpy as np
from pftoolbox.tables import query2df
from pftoolbox.fraud import simulation
import matplotlib.pyplot as plt
import datetime as dt
import seaborn as sns
import click
from dateutil.relativedelta import relativedelta

%run db_access.py
%run credenciales.py


¡BIENVENIDO, GUARDIÁN DEL ECOSISTEMA!



Aclaraciones: 
- Pensado para cortes más restrictivos esto, no para flexibilizaciones. Se necesita suponer por ejemplo que el incoming de nuevos grupos que hoy se rechaza es 100% por ejemplo

- Considero solo flow.type: MI . En redshift no hace falta esta aclaración
- PAY_PM_TYPE_ID  in ('credit_card','debit_card')
- Considero solo config_id: 'OFF','GTW','REC'. En redshift trae esto, sería no traer STD ni 'MIS'
- Considero solo flujo aprobado (A y D) 
- Tomo de base en la join a la tabla de Tera
- Redshift nose porque, en los días limites hay pagos que no trae, por eso le pido que en redshift, traiga un dia de mas y un día de menos, por lo que siempre, redshift debería tener algunos pagos de mas
- Redshift suele actualizar los pagos un poco más tarde, por lo que hay pagos del día de ayer que puede que no esten en Redshift
CON ESTAS REGLAS, REDSHIFT SIEMPRE DEBERÍA TRAER 2 DIAS DE PAGOS DE MÁS. Nose porque, a veces aleatoreamente teradata con esta logica trae menos pagos... igual es menos de 0,5%

INPUT 1

In [2]:
# Seller=click.prompt("Ingresar nombre del Seller")
# desde = click.prompt("Desde", default=(dt.datetime.now()+relativedelta(months=-2)).strftime("%Y-%m-%d"))
# hasta=click.prompt("Hasta", default=(dt.datetime.now()).strftime("%Y-%m-%d"))
# Reintentos=click.prompt("Se trabaja con o sin reintentos?", default='Sin')

In [3]:
#Defino variables
desde='2021-07-15'  #'2020-10-01'
hasta='2021-08-18'
Seller='419721672' #pichau

In [4]:
#Nose porque este campo: spr.creation_date trae menos campos y tengo que extenderle los limites
desde2=((pd.to_datetime(desde))+dt.timedelta(days=-1)).strftime("%Y-%m-%d")
hasta2=((pd.to_datetime(hasta))+dt.timedelta(days=1)).strftime("%Y-%m-%d")

In [5]:
Reintentos='Con'  #'Sin'/'Con'
Regla_En='Redshift' #Redshift/Teradata
# Periodo='dia'    #/dias/semana/mes'

En caso de querer trabajar sobre un flujo puntual, llamar "regla" al flujo como variable, no como filtro! y se trabajará sobre cuando la regla es==1. Si se define en Redshift, aclararlo arriba

In [6]:
reglaTera=''
reglaRedshift="""case when datediff(day, dev.creation_Date, so.creation_date) < 7then 0 else 1 end regla,""" #and so.has_approved_payment_tc='N' 

# --Reglas REDSHIFT
# --case when datediff(day, dev.creation_Date, so.creation_date) < 7 and datediff(day, cus_sen.site_since, so.creation_date) > 7 and pp.mp_prod_id in ('BC32A1FTRPP001U8NHFG','BF6T4TL5G6HG01KPO2LG','BD72KDPQIJ2G01BAK93G','BC32BHVTRPP001U8NHL0','BJEO9TFBF6RG01IIIOU0','BC32CANTRPP001U8NHO0','BC32BQJU643001OI397G','BJDGPHVBF6RG01IIIOSG','BF3L8CCEVKKG01NFMI70','BEL8E9E0CAH001G99FI0','BF3L7UNP2P4001H0L200','BT7OFH09QS3001K5A0H0','BT7OEN7EOO6G01NJK3Q0','BSOO1O89QS3001K5A0GG','BC32CCRU643001OI39AG','BCT80R50FCR001F95260','BEVP8TU8QCDG01NJH370','BT7OE9FEOO6G01NJK3PG','BF3L7V7P2P4001H0L21G','BT7OF5FEOO6G01NJK3QG','BC32A7VTRPP001U8NHK0','BF3L8C7P2P4001H0L220','BC329V3U643001OI3910','BFRFF6JDNCEG02A7IS1G','BJDGOQNBF6RG01IIIORG','BSOO28NEOO6G01NJK3O0','BJEO9NVBF6RG01IIIOTG','BF3L7UVP2P4001H0L210','BF3L7V4EVKKG01NFMI6G','BC32A7RU643001OI3940','BC32A4RU643001OI3930','BC32A7VTRPP001U8NHJ0','BC32A57TRPP001U8NHHG','BC32C7VTRPP001U8NHNG','BRP4K1BTEN2001J1IEH0','BC32CPFTRPP001U8NHRG','BF3JOAKEVKKG01NFMI1G','BF3L7V4EVKKG01NFMI60') then 0 else 1 end regla,

# --Reglas SOR
# --to_char(case when ( sor.creation_date <= dev.frd_device_ml_creation_date + interval '7' day) then 1 else 0 END) regla,
# --prod2.MP_PROD_USER_TYPE='guest'and sor.creation_date <= dev.frd_device_ml_creation_date + interval '15' day and FRD_HAS_APPROVED_PAYMENT_TC='N' and  op_dol_amount >50
# --to_char(case when (prod2.MP_PROD_USER_TYPE='guest'and sor.creation_date <= dev.frd_device_ml_creation_date + interval '15' day and FRD_HAS_APPROVED_PAYMENT_TC='N' and  op_dol_amount >50) then 1 else 0 END) regla,
# --to_char(case when((cast(sor.creation_date as date format 'yyyy-mm-dd') - cast(dev.FRD_DEVICE_ML_CREATION_DATE as date format 'yyyy-mm-dd'))<7) then 1 else 0 end) regla,

In [14]:
# Me traigo try last y la notificacion de cbks

query01= """
Select
"""
query010="""
sor.sc_cust_id,
sor.pay_payment_id,
sor.pay_try_last,
sor.flag_notif_cbk,
sor.site_id,
sor.pcc_status,
sor.stc_profile_id,
to_char(sor.risk) TeraRisk,
to_char(sor.creation_date ,'yyyy-mm-dd-hh') creationdate,
(case when (cbk.FRD_CST_DISPUTE_STATUS='WON' or (cbk.FRD_CST_DISABLE_BUY_MP=0 and not cbk.FRD_CST_NEGOT_STAT_BUY_ID = 'TKO')) then 1 ELSE 0 END ) as "cbk transac Hispanos",
cbk.FRD_CST_FRAUD_STAT_BUY_ID as "cbk trasnc MLB"
from SCORING.scoring_origin_report_all sor
left join WHOWNER.BT_FRAUD_CBK_CASES cbk on cbk.payment_id=sor.pay_payment_id --MAS DETALLE GESITÓN CBK
left join WHOWNER.BT_MP_PRODUCTS prod2 on prod2.mp_prod_id=sor.mp_prod_id --saber si el pago fue guest
left join WHOWNER.LK_FRAUD_DEVICE dev on dev.pay_payment_id=sor.pay_payment_id --info del device

where 1=1

and sor.pcc_status in ('A','D','I') 
and PAY_PM_TYPE_ID in ('credit_card','debit_card')
and sor.config_id in ('OFF','REC')--,'GTW','REC','MIS') 
and sor.flow_type='MI'

and sor.creation_date>= date'"""
query03="'and sor.creation_date<= date'"
query04= "' and sor.cus_cust_id_sel in ("
query05=") --sample 30000"
query06=query01+reglaTera+query010+desde+query03+hasta+query04+Seller+query05

# df2=query2df.redshift(query2,credenciales,RS_access,exclude_drop=True)
dfT=query2df.teradata(query06,credenciales,TERA_access,exclude_drop=True)

Total:	1 query(ies)
	0 create(s)
	1 select(s)
	0 drop(s) excluded
	0 other(s)


Tiempo:	452.31 segs 

Output:	df con 11135 registros



,SC_CUST_ID,PAY_PAYMENT_ID,PAY_TRY_LAST,FLAG_NOTIF_CBK,site_id,PCC_STATUS,STC_PROFILE_ID,TeraRisk,creationdate,cbk transac Hispanos,cbk trasnc MLB
0,2.700348e+11,1.600191e+10,1,0,MLB,A,MIDHIGH,18,2021-07-23-02,0,None
1,2.697091e+11,1.591949e+10,1,0,MLB,A,LOW,0.03,2021-07-18-04,0,None
2,2.699068e+11,1.596845e+10,1,0,MLB,A,LOW,0.19,2021-07-21-03,0,None
3,2.700938e+11,1.601723e+10,1,0,MLB,A,LOW,0,2021-07-24-12,0,None
4,2.694072e+11,1.590028e+10,1,0,MLB,A,LOW,0.01,2021-07-17-11,0,None


In [16]:
query01b= """
Select
"""

query10= """
so.pay_payment_id,
so.scoring_id,
to_char(so.creation_date,'yyyy-mm-dd') as dia,
st.tree_id,
so.last_node,
node_id,
so.risk,
so.op_dol_amount
from  fraud.public.scoring_origin_mi so
left join scoring.scoring_provider_result spr on so.scoring_id=spr.scoring_id and so.provider_id = spr.scoring_provider
left join scoring.scoring_tree_version st on st.scoring_id = spr.external_id and so.site_id = st.site_id
left join scoring.hypothesis_tree_node htn on htn.tree_id = st.tree_id and htn.tree_version_id = st.tree_version_id and htn.site_id = st.site_id and htn.node_position = trunc(so.last_node/2)
left join fraud.scoring.bt_mp_pay_payments pp on  pp.pay_payment_id = so.pay_payment_id
left join scoring_mi.customers cus_sen on (so.scoring_id = cus_sen.scoring_id and cus_sen.cust_id = so.sender_id)
left join scoring_mi.device_ml dev on (so.scoring_id = dev.scoring_id)
--left join fraud.scoring_mi.smart_id si on si.scoring_id=so.scoring_id  --qty_devices

--left join fraud.scoring_mi.items_off io on so.scoring_id=io.scoring_id 

where 1=1
--and so.pay_payment_id in ()
and so.pcc_status in ('A','D','I')
and so.receiver_id in ("""

query11=""") and so.creation_date>=date '"""
query12="""' and spr.creation_date>=date '"""
query13="""' and st.op_timestamp>=date '"""
query15="""' and so.creation_date<=date '"""
query16="""' and spr.creation_date<=date '"""
query17="""' and st.op_timestamp<=date '"""
query14="'"

query=query01b+reglaRedshift+query10+Seller+query11+desde2+query12+desde2+query13+desde2+query15+hasta2+query16+hasta2+query17+hasta2+query14

dfUR=query2df.redshift(query,credenciales,RS_access,exclude_drop=True)
# dfIn=query2df.teradata(query,credenciales,TERA_access,exclude_drop=True)



Total:	1 query(ies)
	0 create(s)
	1 select(s)
	0 drop(s) excluded
	0 other(s)


InternalError: (psycopg2.errors.InternalError_) Query (19521293) cancelled by WLM abort action of Query Monitoring Rule "others_query_execution_time".
DETAIL:  
  -----------------------------------------------
  error:  Query (19521293) cancelled by WLM abort action of Query Monitoring Rule "others_query_execution_time".
  code:      1078
  context:   Query (19521293) cancelled by WLM abort action of Query Monitoring Rule "others_query_execution_time".
  query:     0
  location:  wlm_query_action.cpp:156
  process:   wlm [pid=45732]
  -----------------------------------------------


[SQL: Select case when datediff(day, dev.creation_Date, so.creation_date) < 7then 0 else 1 end regla, so.pay_payment_id, so.scoring_id, to_char(so.creation_date,'yyyy-mm-dd') as dia, st.tree_id, so.last_node, node_id, so.risk, so.op_dol_amount from  fraud.public.scoring_origin_mi so left join scoring.scoring_provider_result spr on so.scoring_id=spr.scoring_id and so.provider_id = spr.scoring_provider left

In [ ]:
df00=dfUR

In [ ]:
dfIn1=dfT

In [ ]:
dfIn1['FLAG_NOTIF_CBK_NO_TRANSACCIONAL']=dfIn1['FLAG_NOTIF_CBK']

if (dfIn1['site_id'].max()=='MLB'):
    IndexB = dfIn1[dfIn1['cbk trasnc MLB']=='TRANSACTIONAL'].index #Devuelve los index de los valores que estan en Lista
    dfIn1.loc[IndexB,'FLAG_NOTIF_CBK_NO_TRANSACCIONAL'] = 0   #Reemplaza los valores de'Others' por valor en Col Ruta en los index definidos 
else:
    IndexH = dfIn1[dfIn1['cbk transac Hispanos']==1].index #Devuelve los index de los valores que estan en Lista
    dfIn1.loc[IndexH,'FLAG_NOTIF_CBK_NO_TRANSACCIONAL'] = 0   #Reemplaza los valores de'Others' por valor en Col Ruta en los index definidos 

In [ ]:
# x=df00['pay_payment_id'].count()-dfT['PAY_PAYMENT_ID'].count()

# print(df00['pay_payment_id'].count()) #Tera
# print(dfT['PAY_PAYMENT_ID'].count()) #Redshift (CASI SIEMPRE DEBERIA SER MAYOR, PORQUE CONSIDERA UN DIA ANTES Y UN DIA DESPUES, ASI TERA ESTA COMPLETO)
# print(x) #Pagos de más que hay en Tera
# print(round((x/df00['pay_payment_id'].count())*100,2))

In [ ]:
# Uno las dos bases y veo si hay muhcos campos que quedaron sin unirse. Tomo como base a la base de redshift
dfIn1['PAY_PAYMENT_ID']=dfIn1['PAY_PAYMENT_ID'].apply(lambda x: round(x))

if Regla_En=='Redshift':
    df01=pd.merge(dfIn1,df00, left_on='PAY_PAYMENT_ID',right_on='pay_payment_id', how='right') #Tomo de base a tera
else:
    df01=pd.merge(dfIn1,df00, left_on='PAY_PAYMENT_ID',right_on='pay_payment_id', how='left') #Tomo de base a tera
# df01=df01.drop(df01[df01['PAY_TRY_LAST']].isnull().index, inplace=True)

# df01.isnull().sum()

In [ ]:
#Relleno los campos vacios, trabajo solo con pay_try_last
df=df01
df['PAY_TRY_LAST']=df['PAY_TRY_LAST'].fillna(1)
df['FLAG_NOTIF_CBK']=df['FLAG_NOTIF_CBK'].fillna(0)

In [ ]:
df1['FLAG_NOTIF_CBK'].sum()

In [ ]:
#NECESARIO CUANDO NOS INFORMAN CBKS Y NO ESTAN MARCADOS. #2 columnas: payments_id y una columna de 1, los pagos fraudes
try:
    path='C:/Users/mgaona/Desktop/Analisis/base python/fraude_pichau.xlsx'
    p_fraude=pd.read_excel(path) #puede ser read_csv
    p_fraude= pd.DataFrame(p_fraude)
    # p_fraude['PAY_PAYMENT_ID']=p_fraude['PAY_PAYMENT_ID'].apply(lambda x: round(x))
    # p_fraude['payment']=p_fraude['payment'].apply(lambda x: float(x))

    df=pd.merge(df,p_fraude, left_on='PAY_PAYMENT_ID',right_on='PAY_PAYMENT_ID', how='left') 

    # df1.reset_index(inplace = True)
    IndexR = df[df['cbks_']==1].index #Devuelve los index de los valores que estan en Lista
    # df1['FLAG_NOTIF_CBK']='Others' #DEFINE COLUMN
    df.loc[IndexR,'FLAG_NOTIF_CBK'] = df.loc[IndexR,'cbks_'] #Reemplaza los valores de'Others' por valor en Col Ruta en los index definidos 
except:
    pass

In [ ]:
df1['FLAG_NOTIF_CBK'].sum()

# Filtro solo flujo de regla

In [ ]:
try:
    #Filtro solo el grupo que me interesa
    df['regla']=df['regla'].apply(lambda x: int(x))
    pivotR=df.pivot_table(index=['FLAG_NOTIF_CBK'],columns=['regla'],values=['op_dol_amount'],aggfunc=np.sum, margins=True) #para ver cantidad, cambiar sum por         
    pivotR=pivotR['op_dol_amount']
    pivotR.sort_values
    df=df[df['regla']==1]
#     pivotR/pivotR.sum()
    textF='Los resultados son SOLO sobre el flujo de la regla'
    textF='Resultados del flujo de la regla que representa '+str(round(Share_flujo_regla*100,1))+' % el flujo total. El cbk real de la regla representa '+str(round(Share_cbk_real_regla*100,1))+'% del flujo total\n'

except:
        pivotR=0
        print('Es todo el flujo')
        textF='Se analizo todo el flujo del seller (a veces es sobre new device por ejemplo)'


In [ ]:
try:
    pivotR=pivotR.sort_values(by='All',ascending=False)
    Share_flujo_regla=pivotR[1].iloc[0]/pivotR['All'].iloc[0]
    Share_cbk_real_regla=pivotR[1].iloc[2]/pivotR['All'].iloc[2]
except:
    pass


In [ ]:
if (Reintentos=='Sin'):
    df=df[df['PAY_TRY_LAST']==1]

In [ ]:
# dfM[dfM['PAY_PAYMENT_ID']==13799511661] 13759402599

# SIMULAR

In [ ]:
Simu_OK = click.prompt("Quiero Simular? (Revisar campos de abajo)", default='Si')
N_Muestra=click.prompt("Cantidad de pagos", default=df['PAY_PAYMENT_ID'].count())

In [ ]:
description = 'Pichau sin fato'
setteos     = {'tree':         'PRUEBA_TUTE',
               'tree_version': 107,
               'site':         'MLB',
               'flow':         'MI',
               'team':         'prod_bsas'}

In [ ]:
Sim='No'
if Simu_OK=='Si':
    dfM=df
    if int(N_Muestra) != (df['PAY_PAYMENT_ID'].count()):
        dfM=df.sample(n=int(N_Muestra))
    dfM['PAY_PAYMENT_ID2']=dfM['PAY_PAYMENT_ID']
    dfM=dfM.drop(labels='PAY_PAYMENT_ID',axis=1)
    results=simulation.execute(dfM,description,setteos)
    Sim=='Si'
else:
    pass

In [ ]:
if Simu_OK=='Si':
    results['risk']=results['risk']
    #  results['risk']=results.iloc[:,-2:-1]
    results['op_dol_amount']=results['op_dol_amount']
    results['tree_id']=setteos['tree']
    df=results
else:
    pass

# Termina Simulacón

In [ ]:
#Miro de todo el flujo, en que provider/arbol terminó (se entiende entonces que es flujo OFF/GTW, MI, regular)
if Simu_OK=='Si':
    df['tree_id']=df['TREE_ID'].fillna('N/A')
    df['op_dol_amount']=df['OP_DOL_AMOUNT']
    df['node_id']=df['NODE_ID']
    df['risk']=df['RISK']
    #df['PAY_PAYMENT_ID'].apply(lambda x: round(x))
else:
    df['tree_id']=df['tree_id'].fillna('N/A')
    
pivot_0=df.pivot_table(index=['tree_id'],values=['op_dol_amount'],aggfunc=np.size).sort_values(by='op_dol_amount',ascending=False) #para ver cantidad, cambiar sum por         
plt.pie(pivot_0['op_dol_amount'],labels=pivot_0.index,autopct='%.2f')
plt.show()
df['tree_id'].unique()


INPUT 2

In [ ]:
tree = click.prompt("Ingresar el arbol", default=str(pivot_0.index[0]))
percentil=click.prompt("Ingresar cantidad de grupos", default=10)
regla=click.prompt("Ingresar nombre de la regla", default='Sin')

In [ ]:
# tree=input('Seleccionar el Arbol')
# percentil=input('Seleccionar percentil')
# regla=input('Seleccionar la regla')

In [ ]:
# # Miro cuales fueron los nodos de rechazo, según el arbol seleccionado

# tree='ONLINE_PAYMENTS_CA_ALPHA' #Aca hay que elegir
# percentil=10 #multiplo de 100. el % que quiero que tenga cada cuadradito
# regla='Sin'  #'Sin'/'Nombre de la regla'

In [ ]:
texto2='El '+str(int(pivot_0['op_dol_amount'][0]/pivot_0['op_dol_amount'].sum()*100))+' % del flujo es por el arbol '+str(tree)

# df['node_id']=df['node_id'].fillna('Aprobado')
pivot_1=df[(df['PCC_STATUS']=='I')&(df['tree_id']==tree)].pivot_table(index=['node_id'],values=['op_dol_amount'],aggfunc=np.sum).sort_values(by='op_dol_amount',ascending=False)   
plt.pie(pivot_1['op_dol_amount'],labels=pivot_1.index,autopct='%.2f')
plt.show()
df['node_id'].unique()
texto0='El '+str(int(pivot_1['op_dol_amount'][0]/pivot_1['op_dol_amount'].sum()*100))+' % del rechazo es por el nodo '+str(pivot_1.index[0])

In [ ]:
#Acomodo los formatos, y me quedo solo con: el arbol seleccionado arriba, y los pagos con risk<1
# df['op_dol_amount']=df['op_dol_amount'].fillna(0)
df['op_dol_amount']=df['op_dol_amount'].apply(lambda x: float(x))
df=df[df['risk']!=''] #Si no trae RISK, es porque en SrongRules tuvo profile MIDINH O HIGH
df['risk']=df['risk'].fillna(0)
# df['risk']=df['risk'].replace(np.nan,df['TeraRisk']) #reemplasa NaN/None (sin valor) por la media de una columna
df['risk']=df['risk'].apply(lambda x: float(x))
df=df[(df['risk']<=1)&(df['tree_id']==tree)]

In [ ]:
if regla=='Sin':
    dft=df
else:
    dft=df
    dft[regla]=dft[regla].apply(lambda x: int(x))
    dft=dft[dft[regla]==1]

In [ ]:
dft['class_risk']=0
dft['class_op_dol_amount']=0

In [ ]:
# dtr=dft.iloc[:, 0:3] #solución? hay 2 columnas de payment_id..por ahi
# dft.rename(columns={'PAY_PAYMENT_ID':'PAY_PAYMENT_ID2'})
# dft.rename(index={1:'PAY_PAYMENT_ID'})


In [ ]:
#HACERLO CON INDEX ASI SE HACE MAS RAPIDO!

# #CREO COLUMNAS PARA DIVIDIR POR PERCENTIL DE VALORES A LOS MONTOS

total=dft['op_dol_amount'].sum()
dft=dft.sort_values(by='op_dol_amount')
dft['op_dol_amount%']=dft['op_dol_amount']/total
dft['op_dol_amount%Dist']=0
for i in range(int(dft['PAY_PAYMENT_ID'].count())):
    if i==0:
        dft['op_dol_amount%Dist'].iloc[i]=dft['op_dol_amount%'].iloc[i]
    else:
        dft['op_dol_amount%Dist'].iloc[i]=dft['op_dol_amount%'].iloc[i]+dft['op_dol_amount%Dist'].iloc[i-1]

for r in range(0,percentil):
    if r==percentil-1:
        Index = dft[(dft['op_dol_amount%Dist']>=r/percentil)].index 
        dft.loc[Index,'class_op_dol_amount'] = round(dft['op_dol_amount'][(dft['op_dol_amount%Dist']>(0.999))].iloc[0],1)
    else:
        Index = dft[(dft['op_dol_amount%Dist']>r/percentil)&(dft['op_dol_amount%Dist']<=(r+1)/percentil)].index 
        dft.loc[Index,'class_op_dol_amount'] = round(dft['op_dol_amount'][(dft['op_dol_amount%Dist']>((r+1)/percentil))&(dft['op_dol_amount%Dist']<=((r+1)/percentil+0.1))].sort_values().iloc[0],1)        
        
            
total=dft['op_dol_amount'].sum()
dft=dft.sort_values(by='risk')
dft['op_dol_amount%2']=dft['op_dol_amount']/total
dft['op_dol_amount%Dist2']=0

for i in range(int(dft['PAY_PAYMENT_ID'].count())):
    if i==0:
        dft['op_dol_amount%Dist2'].iloc[i]=dft['op_dol_amount%2'].iloc[i]
    else:
        dft['op_dol_amount%Dist2'].iloc[i]=dft['op_dol_amount%2'].iloc[i]+dft['op_dol_amount%Dist2'].iloc[i-1]

for j in range(0,percentil):
    if j==percentil-1:
        Index = dft[(dft['op_dol_amount%Dist2']>=j/percentil)].index 
        dft.loc[Index,'class_risk'] = round(dft['risk'][(dft['op_dol_amount%Dist2']>(0.999))].iloc[0],4)
    else:
        Index = dft[(dft['op_dol_amount%Dist2']>j/percentil)&(dft['op_dol_amount%Dist2']<=(j+1)/percentil)].index 
        dft.loc[Index,'class_risk'] = round((dft['risk'][(dft['op_dol_amount%Dist2']>((j+1)/percentil))&(dft['op_dol_amount%Dist2']<=((j+1)/percentil+0.1))].sort_values()).iloc[0],4)


In [ ]:
#Tener el valor máximo, despues ir sumando hasta llegar al valor 10%, del 20% y asi.

In [ ]:
# Encuentro los percentiles del riesgo y del Amount #cantidades
# rangeM=[]
# rangeR=[]
# for i in range (1,round(100/percentil)+1):
#     rangeM.append(np.percentile(dft['op_dol_amount'],percentil*i)) #percentile

# for j in range (1,round(100/percentil)+1):
#     rangeR.append(np.percentile(dft['risk'],percentil*j))

In [ ]:
#Creo las columnas de los rangos de riesgo y amount


# OPCION 1 GRUPOS DE MONTOS
# dft['class_op_dol_amount']=0

# for r in range(0,round(100/percentil)):
#     if r==0:
#         Index = dft[(dft['op_dol_amount']<=rangeM[r])].index 
#         dft.loc[Index,'class_op_dol_amount'] = round(rangeM[r],0)
#     elif r==10:
#         Index = dft[(dft['op_dol_amount']>rangeM[r-1])].index 
#         dft.loc[Index,'class_op_dol_amount'] = round(rangeM[r],0)
#     else:
#         Index = dft[(dft['op_dol_amount']<=rangeM[r])&(dft['op_dol_amount']>rangeM[r-1])].index 
#         dft.loc[Index,'class_op_dol_amount'] = round(rangeM[r],0)



# dft['class_risk']=0

# for p in range(0,round(100/percentil)):
#     if p==0:
#         Index = dft[(dft['risk']<=rangeR[p])].index 
#         dft.loc[Index,'class_risk'] = round(rangeR[p],4)
#     elif p==10:
#         Index = dft[(dft['risk']>rangeR[p-1])].index 
#         dft.loc[Index,'class_risk'] =  round(rangeR[p],4) 
#     else:
#         Index = dft[(dft['risk']<=rangeR[p])&(dft['risk']>rangeR[p-1])].index 
#         dft.loc[Index,'class_risk'] =  round(rangeR[p],4)


In [ ]:
#Creo todas las tablas que me interesan

In [ ]:
# dfw = pd.DataFrame(pivot_)
# dfw.sort_index()

In [ ]:
#Distribución de IncomingPF por Amount
pivot_=dft.pivot_table(index=['class_risk'],columns=['class_op_dol_amount'],values=['op_dol_amount'],aggfunc=np.sum, fill_value=0,margins=True)#para ver cantidad, cambiar sum por 
pivot_=pivot_['op_dol_amount']
IncomingTot=pivot_.sum().sum()/4

#Distribución de IncomingPF por QTY
pivot_2=dft.pivot_table(index=['class_risk'],columns=['class_op_dol_amount'],values=['op_dol_amount'],aggfunc=np.size, fill_value=0,margins=True) #para ver cantidad, cambiar sum por 
pivot_2=pivot_2['op_dol_amount']

In [ ]:
diccMonto={}
f=1
for i in pivot_.columns:
    diccMonto[i]=f
    f=f+1
diccRisk={}

t=1
for i in pivot_.index:
    diccRisk[i]=t
    t=t+1


In [ ]:
#Distribución de IncomingPF MIDHIGH  por amount
pivot_IMIDHIGH=dft[(dft['STC_PROFILE_ID']=='MIDHIGH')].pivot_table(index=['class_risk'],columns=['class_op_dol_amount'],values=['op_dol_amount'],aggfunc=np.sum, fill_value=0,margins=True) #para ver cantidad, cambiar sum por                 
pivot_IMIDHIGH=pivot_IMIDHIGH['op_dol_amount']

#Distribución de TPV por amount
pivot_A=dft[(dft['PCC_STATUS']=='A')|(dft['PCC_STATUS']=='D')].pivot_table(index=['class_risk'],columns=['class_op_dol_amount'],values=['op_dol_amount'],aggfunc=np.sum, fill_value=0,margins=True) #para ver cantidad, cambiar sum por 
pivot_A=pivot_A['op_dol_amount']
Mont_Aprob_corteInicial=pivot_A.sum().sum()/4

In [ ]:
#Distribución de TPV LOW por QTY
pivot_AQ=dft[((dft['PCC_STATUS']=='A')|(dft['PCC_STATUS']=='D'))&(dft['STC_PROFILE_ID']=='LOW')].pivot_table(index=['class_risk'],columns=['class_op_dol_amount'],values=['op_dol_amount'],aggfunc=np.size, fill_value=0,margins=True) #para ver cantidad, cambiar sum por 
pivot_AQ=pivot_AQ['op_dol_amount']

#Distribución de TPV LOW  por amount
pivot_ALOW=dft[((dft['PCC_STATUS']=='A')|(dft['PCC_STATUS']=='D'))&(dft['STC_PROFILE_ID']=='LOW')].pivot_table(index=['class_risk'],columns=['class_op_dol_amount'],values=['op_dol_amount'],aggfunc=np.sum, fill_value=0,margins=True) #para ver cantidad, cambiar sum por         
pivot_ALOW=pivot_ALOW['op_dol_amount']
Mont_AprobLOW_corteInicial=pivot_ALOW.sum().sum()/4

In [ ]:
#Distribución de TPV MIDHIGH  por amount
try:
    pivot_AMIDHIGH=dft[((dft['PCC_STATUS']=='A')|(dft['PCC_STATUS']=='D'))&(dft['STC_PROFILE_ID']=='MIDHIGH')].pivot_table(index=['class_risk'],columns=['class_op_dol_amount'],values=['op_dol_amount'],aggfunc=np.sum, fill_value=0,margins=True) #para ver cantidad, cambiar sum por                 
    pivot_AMIDHIGH=pivot_AMIDHIGH['op_dol_amount']
except:
    pivot_AMIDHIGH=0

In [ ]:

#Niveles de cbk por grupo
try:
    pivot_cbk=dft[dft['FLAG_NOTIF_CBK']==1].pivot_table(index=['class_risk'],columns=['class_op_dol_amount'],values=['op_dol_amount'],aggfunc=np.sum, fill_value=0,margins=True) #para ver cantidad, cambiar sum por 
    pivot_cbk=pivot_cbk['op_dol_amount']
    DistrCbkusd=(pivot_cbk.sum().sum()/4)
    DistrCbkTot=pivot_cbk*100/DistrCbkusd
except:
    pivot_cbk=0
    
#Niveles de cbk LOW por grupo
try:
    pivot_cb82=dft[(dft['FLAG_NOTIF_CBK']==1)&(dft['STC_PROFILE_ID']=='LOW')].pivot_table(index=['class_risk'],columns=['class_op_dol_amount'],values=['op_dol_amount'],aggfunc=np.sum, fill_value=0,margins=True) #para ver cantidad, cambiar sum por 
    pivot_cb82=pivot_cb82['op_dol_amount']
    DistrCbkLOWusd=(pivot_cb82.sum().sum()/4)
    DistrCbkLow=pivot_cb82*100/DistrCbkLOWusd

except:
    pivot_cb82=0

In [ ]:
#QTY cbks
try:
    pivot_cbk0=dft[dft['FLAG_NOTIF_CBK']==1].pivot_table(index=['class_risk'],columns=['class_op_dol_amount'],values=['op_dol_amount'],aggfunc=np.size, fill_value=0,margins=True) #para ver cantidad, cambiar sum por 
    pivot_cbk0=pivot_cbk0['op_dol_amount']
except:
    pivot_cbk0=0

#QTY cbks LOW
try:
    pivot_cbk01=dft[(dft['FLAG_NOTIF_CBK']==1)&(dft['STC_PROFILE_ID']=='LOW')].pivot_table(index=['class_risk'],columns=['class_op_dol_amount'],values=['op_dol_amount'],aggfunc=np.size, fill_value=0,margins=True) #para ver cantidad, cambiar sum por 
    pivot_cbk01=pivot_cbk01['op_dol_amount']
except:
    pivot_cbk01=0

In [ ]:
# # CUANDO HAY UNA FILA SIN VALORES en classmonto, ACA SE LA AGREGA CON ALGUNOS VALORES NULL... A REVISAR!! 
# u=0
# for x in list(diccMonto.keys())[:-1]:
#     r=10
#     for i in list(dft['class_op_dol_amount'][(dft['FLAG_NOTIF_CBK_NO_TRANSACCIONAL']==1)&(dft['STC_PROFILE_ID']=='LOW')].unique()):
#         if x==i:
#             r=r-1
#         else:
#             pass
    
#     if r==10: #ACA CAMBIAR EL 10!!
#         u=u+1
#         dft=dft.append({'class_risk' : dft['class_risk'].unique()[0] ,'STC_PROFILE_ID':'LOW','FLAG_NOTIF_CBK_NO_TRANSACCIONAL':1,'FLAG_NOTIF_CBK':1,'op_dol_amount':0.0002,'class_op_dol_amount':x},ignore_index=True)

# # dft[dft['FLAG_NOTIF_CBK_NO_TRANSACCIONAL']==1].tail(10)

# # CUANDO HAY UNA FILA SIN VALORES, ACA SE LA AGREGA CON ALGUNOS VALORES NULL... A REVISAR!! 
# for x in list(diccRisk.keys())[:-1]:
#     r=10
#     for i in list(dft['class_risk'][(dft['FLAG_NOTIF_CBK_NO_TRANSACCIONAL']==1)&(dft['STC_PROFILE_ID']=='LOW')].unique()):
#         if x==i:
#             r=r-1
#         else:
#             pass
    
#     if r==10: #ACA CAMBIAR EL 10!!
#         u=u+1
#         dft=dft.append({'class_risk' : x ,'STC_PROFILE_ID':'LOW','FLAG_NOTIF_CBK_NO_TRANSACCIONAL':1,'FLAG_NOTIF_CBK':1,'op_dol_amount':0.0002,'class_op_dol_amount':dft['class_op_dol_amount'].unique()[0]},ignore_index=True)

# dft[dft['FLAG_NOTIF_CBK_NO_TRANSACCIONAL']==1].tail(10)

# print('Se creaeron',u,'cbks para que haya 10 filas y 10 columnas')


# OPCION 2. EN LA DATAFREMA AGREGAR LA FILA! PERO TENGO LIO PARA ORDENAR CON LA COLUMNA ALL...

# df_Aux={1:[0] ,2:[0],3:[0]}
# df_Aux = pd.DataFrame(df_Aux)     
# df_Aux=df_Aux.reindex([0.104], axis=0)
# w=pd.concat([DistrCbkLsT, df_Aux]).fillna(0)
# # w=w.drop(['All'], axis=0) # Drop columns Temporaly. DROP (Columnas no tienen index!)
# # w.sort_index(axis=0,kind='heapsort')


In [ ]:
#Distribución de cbk por amount
try:
    pivot_cbkF=dft[dft['FLAG_NOTIF_CBK_NO_TRANSACCIONAL']==1].pivot_table(index=['class_risk'],columns=['class_op_dol_amount'],values=['op_dol_amount'],aggfunc=np.sum, fill_value=0,margins=True) #para ver cantidad, cambiar sum por 
    pivot_cbkF=pivot_cbkF['op_dol_amount']
except:
    pivot_cbkF=0
    
#QTY cbks transaccional
try:
    pivot_cbk0F=dft[dft['FLAG_NOTIF_CBK_NO_TRANSACCIONAL']==1].pivot_table(index=['class_risk'],columns=['class_op_dol_amount'],values=['op_dol_amount'],aggfunc=np.size, fill_value=0,margins=True) #para ver cantidad, cambiar sum por 
    pivot_cbk0F=pivot_cbk0F['op_dol_amount']
except:
    pivot_cbk0F=0

#QTY cbks LOW transaccional
try:
    pivot_cbk01F=dft[(dft['FLAG_NOTIF_CBK_NO_TRANSACCIONAL']==1)&(dft['STC_PROFILE_ID']=='LOW')].pivot_table(index=['class_risk'],columns=['class_op_dol_amount'],values=['op_dol_amount'],aggfunc=np.size, fill_value=0,margins=True) #para ver cantidad, cambiar sum por 
    pivot_cbk01F=pivot_cbk01F['op_dol_amount']
    
except:
    pivot_cbk01F=0
    
# cbks LOW y sin ser trnasaccionales en monto
try:
    pivot_cbk01T=dft[(dft['FLAG_NOTIF_CBK_NO_TRANSACCIONAL']==1)&(dft['STC_PROFILE_ID']=='LOW')].pivot_table(index=['class_risk'],columns=['class_op_dol_amount'],values=['op_dol_amount'],aggfunc=np.sum, fill_value=0,margins=True) #para ver cantidad, cambiar sum por 
    pivot_cbk01T=pivot_cbk01T['op_dol_amount']
    CbkbTotLow_Tran=pivot_cbk01T.sum().sum()/4
    DistrCbkLsT=pivot_cbk01T*100/CbkbTotLow_Tran
    
    
except:
    DistrCbkLsT=0

In [ ]:
pivot_cbkprofile=dft[dft['PCC_STATUS']=='A'].pivot_table(index=['FLAG_NOTIF_CBK'],columns=['STC_PROFILE_ID'],values=['op_dol_amount'],aggfunc=np.sum,margins=True) #para ver cantidad, cambiar sum por                 

w1=pivot_cbkprofile.iloc[1,0]/(pivot_cbkprofile.iloc[2,0])
w3=pivot_cbkprofile.iloc[1,2]/(pivot_cbkprofile.iloc[2,2])
try:
    w2=pivot_cbkprofile.iloc[1,1]/(pivot_cbkprofile.iloc[2,1])
    texto4= 'Cbk total:'+str(round(w3*100,1))+'% donde el LOW es '+str(round(w1*100,1))+'% y el MIDHIGH '+str(round(w2*100,1))+'%'
except:
    texto4='Cbk total:'+str(round(w3*100,1))+'% donde el LOW es '+str(round(w1*100,1))+'%'


In [ ]:
#Niveles de Cbks LOW con transaccionales por grupo
AcbkTL=pivot_cb82.div(pivot_ALOW, level=1)
AcbkTL=AcbkTL.apply(lambda x: round((x * 100),2))

In [ ]:
#Niveles de Cbks LOW sin ser transaccionales por grupo
AcbkT=pivot_cbk01T.div(pivot_ALOW, level=1)
AcbkT=AcbkT.apply(lambda x: round((x * 100),2))

In [ ]:
#Distribución InocmingPF por Amount en %
pivot_12=(pivot_/(dft['op_dol_amount'].sum()))
s=pivot_12.apply(lambda x: round((x * 100),3))

In [ ]:
# Niveles de Aprobación por grupo
A=pivot_A.div(pivot_, level=1, fill_value=0)
AA=A.apply(lambda x: round((x * 100),0))
AA      

# Niveles de aprobación LOW por grupo
ALOW=pivot_ALOW.div(pivot_, level=1, fill_value=0)
AALOW=ALOW.apply(lambda x: round((x * 100),3))
AALOW 

# Niveles de aprobaicón MIDHIGH por grupo
try:
    AMIDHIGH=pivot_AMIDHIGH.div(pivot_IMIDHIGH, level=1, fill_value=0)
    AAMIDHIGH=AMIDHIGH.apply(lambda x: round((x * 100),3))
    AAMIDHIGH
except:
    AAMIDHIGH=0
    
# Niveles de %cbks por grupo
try:
    Acbk=pivot_cbk.div(pivot_A, level=1)
    Acbk=Acbk.apply(lambda x: round((x * 100),2))
    Acbk
except:
    Acbk=0

# # plt.figure(figsize=[16,4]) # create a new figure. CREATE. If note, you will modify last figure
# f, ((ax1, ax2),(ax3,ax4)) = plt.subplots(2,2, figsize=(30,15))
# sns.heatmap(AA,annot=True, ax=ax1)
# sns.heatmap(AALOW,annot=True, ax=ax2)
# try:
#     sns.heatmap(AAMIDHIGH,annot=True, ax=ax3)
# except:
#     print('sin midhigh aprobado')
# try:
#     sns.heatmap(Acbk,annot=True, ax=ax4)
# except:
#     print('NO hay cbks en este periodo')

# if (dft['PAY_TRY_LAST'].min()==0):
#     Rein='con reintentos'
# else:
#     Rein='sin reintenos'
    

# print('Comparación de riesgo/aprobación desde',desde,'hasta',hasta,'SOLO en el arbol',tree,Rein,'\n\n')
# # sns.set(font_scale=2)
# print('En cada grupo se ve el valor maximo al que pertenece. \n')
# s

In [ ]:
# DistrCbkLsT.size()
Aux0=np.zeros((11,11)) #Zeros matrix
Aux0=pd.DataFrame(Aux0)
Aux0=Aux0.reindex(list(s.index),axis=0)
Aux0=Aux0.reindex(list(s.columns),axis=1)
Aux0=Aux0.fillna(0)
DistrCbkLsT=Aux0+DistrCbkLsT
DistrCbkLsT=DistrCbkLsT.fillna(0)

AcbkT=Aux0+AcbkT
AcbkT=AcbkT.fillna(0)

pivot_cbk01T=Aux0+pivot_cbk01T
pivot_cbk01T=pivot_cbk01T.fillna(0)

AcbkTL=Aux0+AcbkTL
AcbkTL=AcbkTL.fillna(0)

pivot_A=Aux0+pivot_A
pivot_A=pivot_A.fillna(0)

DistrCbkLow=DistrCbkLow+Aux0
DistrCbkLow=DistrCbkLow.fillna(0)
pivot_cb82=Aux0+pivot_cb82
pivot_cb82=pivot_cb82.fillna(0)


pivot_cbk0=Aux0+pivot_cbk0
pivot_cbk0=pivot_cbk0.fillna(0)

pivot_cbk01=Aux0+pivot_cbk01
pivot_cbk01=pivot_cbk01.fillna(0)


# RESULTADO

In [ ]:
print(textF)
print('IncomingPF del arbol (usd):',str(round(int(IncomingTot),0)),'\n TPV del arbol (usd):',str(round(int(Mont_AprobLOW_corteInicial),0)),'\n CBK del arbol (usd):',str(int(round(DistrCbkusd,0))),'\n CBK LOW del arbol (usd):',str(int(round(DistrCbkLOWusd,0))),'\n\n TPN:'
      +str(int(pivot_2.sum().sum()/4)),'\n CBK (qty):',str(int(dfIn1['FLAG_NOTIF_CBK'].sum())),'\n CBK LOW sin Tran(qty):',
      str(int(dfIn1['FLAG_NOTIF_CBK_NO_TRANSACCIONAL'][dfIn1['STC_PROFILE_ID']=='LOW'].sum().sum())),'\n CKBK transac(qty):',
    str(int(dfIn1['FLAG_NOTIF_CBK_NO_TRANSACCIONAL'].sum())),'\n CBK LOW (qty):',str(int(dfIn1['FLAG_NOTIF_CBK'][dfIn1['STC_PROFILE_ID']=='LOW'].sum().sum())),'\n')

# print(texto3,'Se tomo de base a',Regla_En)
print(texto2,'.Se trabajo solo sobre este flujo')

# print(texto5)
plt.title('Nodos de rechazo en el arbol '+tree)
plt.pie(pivot_1['op_dol_amount'],labels=pivot_1.index,autopct='%.2f')
plt.show('porcentajes de pagos aprobados, rechazados, etc.')
plt.show('Configuración')


In [ ]:
f, ((ax1, ax2,ax3,ax4)) = plt.subplots(4, figsize=(20,20))
sns.heatmap(s,annot=True, ax=ax1,vmax=3,robust=True,cmap="YlGnBu")  #Distribución del Incoming PF
sns.heatmap(AALOW,annot=True, ax=ax2,robust=True,cmap="Blues")  #Nivel aprobación LOW
sns.heatmap(DistrCbkLow,annot=True, ax=ax3,vmax=5,robust=True,cmap="YlGnBu") #Distribución del cbk LOW sin transaccional por grupo. DistrCbkTot(tot)/DistrCbkLow (Low)/DistrCbkLsT (LOW y transacc)
sns.heatmap(AcbkTL,annot=True, ax=ax4,cmap="Blues",robust=True) # %AcbkTL=cbks LOW/ AcbkT=cbk LOW sin transaccional 
ax1.title.set_text('Distribución del Incoming PF')
ax2.title.set_text('Nivel de aprobación LOW')
ax3.title.set_text('Distribución del cbk LOW (suma es % de Captura)')
ax4.title.set_text('Nivel de cbks LOW (promedio es % de Eficiencia)')
f. tight_layout(pad=4.0)
plt.show()

In [ ]:
 print(list(s.index)[:-1])
print(list(s. columns)[:-1])

# INPUT NUEVO CORTE

In [ ]:
#Empieza desde monto 0 y riesgo 0

Riesgo_Hasta1= 0.0198
Monto_Hasta1= 2226.1 #Lo incluye

Riesgo_Hasta2=0
Monto_Hasta2=0 #Monto Hasta

Riesgo_Hasta3=0 #Sino poner 0
Monto_Hasta3=0 #Sino poner 0

Riesgo_Hasta4=0 #Sino poner 0
Monto_Hasta4=0  #Sino poner 0


In [ ]:
#CALCULO DE TPV APROBADO NUEVO CORTE (Mismo tpv con mas restricciones)

g1=pivot_A.iloc[0:diccRisk[Riesgo_Hasta1],0:diccMonto[Monto_Hasta1]] #filas & columnas
Sg1=g1.sum().sum()
AprobCorte=Sg1

try:
    g2=pivot_A.iloc[0:diccRisk[Riesgo_Hasta2],diccMonto[Monto_Hasta1]:diccMonto[Monto_Hasta2]] #filas & columnas
    Sg2=g2.sum().sum()
    AprobCorte=AprobCorte+Sg2
except:
    print('Sin bloque 2')

try:
    g3=pivot_A.iloc[0:diccRisk[Riesgo_Hasta3],diccMonto[Monto_Hasta2]:diccMonto[Monto_Hasta3]] #filas & columnas
    Sg3=g3.sum().sum()
    AprobCorte=AprobCorte+Sg3
except:
    print('Sin bloque 3')
try:
    g4=pivot_A.iloc[0:diccRisk[Riesgo_Hasta4],diccMonto[Monto_Hasta3]:diccMonto[Monto_Hasta4]] #filas & columnas
    Sg4=g4.sum().sum()
    AprobCorte=AprobCorte+Sg4
except:
    print('Sin bloque 4')
    
AprobCortePorc=AprobCorte/IncomingTot
AprobCorteTeo=Mont_AprobLOW_corteInicial/IncomingTot

In [ ]:
#CALCULO DE CBK APROBADO CON  NUEVO CORTE

h1=pivot_cb82.iloc[0:diccRisk[Riesgo_Hasta1],0:diccMonto[Monto_Hasta1]] #filas & columnas
Sh1=h1.sum().sum()
CbkbCorte=Sh1

try:
    h2=pivot_cb82.iloc[0:diccRisk[Riesgo_Hasta2],diccMonto[Monto_Hasta1]:diccMonto[Monto_Hasta2]] #filas & columnas
    Sh2=h2.sum().sum()
    CbkbCorte=CbkbCorte+Sh2
except:
    print('Sin bloque 2')

try:
    h3=pivot_cb82.iloc[0:diccRisk[Riesgo_Hasta3],diccMonto[Monto_Hasta2]:diccMonto[Monto_Hasta3]] #filas & columnas
    Sh3=h3.sum().sum()
    CbkbCorte=CbkbCorte+Sh3
except:
    print('Sin bloque 3')
try:
    h4=pivot_cb82.iloc[0:diccRisk[Riesgo_Hasta4],diccMonto[Monto_Hasta3]:diccMonto[Monto_Hasta4]] #filas & columnas
    Sh4=h4.sum().sum()
    CbkbCorte=CbkbCorte+Sh4
except:
    print('Sin bloque 4')
    
CbkCortePorc=CbkbCorte/Mont_AprobLOW_corteInicial
CbkInicial=DistrCbkLOWusd/Mont_AprobLOW_corteInicial

# RESULTADO NUEVO CORTE

In [ ]:
print( textF,'\n')
print('Nivel de aprobación LOW  del arbol',tree,':',round(AprobCorteTeo*100,1),'pp')
print('Nivel de aprobación LOW del arbol',tree,'con nuevo corte:',round(AprobCortePorc*100,1),'pp')
VarAprobPF=AprobCortePorc-AprobCorteTeo
print('Variación de aprobación LOW con nuevo corte:',round((VarAprobPF)*100,2),'pp(',round((AprobCortePorc/AprobCorteTeo-1)*100,1),'%). Si simulo pagos LOW del arbol, se espera nivlel de aprob de',100+round((AprobCortePorc/AprobCorteTeo-1)*100,1),'pp o menos si es que en otros nodos luego lo rechazan\n. La caida es la comentada. el nivel de aprobación que va a quedar es la comentada+ rechazo strong rules\n')

print('Nivel de CBK LOW real sobre TPV:',round(CbkInicial*100,1),'pp')
print('Nivel de Cbk LOW sobre TPV en nuevo corte:',round(CbkCortePorc*100,1),'pp')
print('Variación de Cbk LOW sore TPV en nuevo corte:',round((CbkCortePorc-CbkInicial)*100,2),'pp (',round((CbkCortePorc/CbkInicial-1)*100,1),'%)\n')

CbksIncg=DistrCbkLOWusd/IncomingTot
Nuevo_CbksIncg=CbkbCorte/IncomingTot
Var_CbksIncg=Nuevo_CbksIncg-CbksIncg
# print('Nivel de CBK LOW real sobre IncomingPF:',round(CbksIncg*100,1),'pp')
# print('Nivel de CBK LOW real sobre IncomingPF nuevo corte:',round(Nuevo_CbksIncg*100,1),'pp ')
# print('Variación de CBK LOW real sobre IncomingPF nuevo corte:',round((Var_CbksIncg)*100,2),'pp \n')

# print('EFICIENCIA (con cbk estimado):',round(/(Mont_Aprob_corteInicial-AprobCorte)*100,1),'% (Sobre el TPV LOW rechazado x nuevo corte)') #Del Incoming rechazado, que % es Cbk (falso positivo lo que falta para llegar a 100%)
print('EFICIENCIA (con cbk real. estimado es mayor):',round((DistrCbkLOWusd-CbkbCorte)/(Mont_Aprob_corteInicial-AprobCorte)*100,1),'% (Sobre el TPV LOW rechazado x nuevo corte)') #Del Incoming rechazado, que % es Cbk (falso positivo lo que falta para llegar a 100%)
print('CAPTURA DEL CBK:',round((DistrCbkLOWusd-CbkbCorte)/DistrCbkLOWusd*100,1),'% (Del total del cbk LOW, cuanto se captura con la nueva regla)')
print('RIFP:',round((VarAprobPF-Var_CbksIncg)/Var_CbksIncg,2),'pp (Cuanto aumenta el rechazo HR del Falso Positivo por capturar 1pp de cbk)')



In [ ]:
print('Idealmente combinar este nuevo corte con el viejo')
print('{\nIF([#OP_DOL_AMOUNT >=',Monto_Hasta1,'] AND [#RISK >',Riesgo_Hasta1,']){@VERDADERO;}\nIF([#OP_DOL_AMOUNT >',Monto_Hasta1,'] AND [#OP_DOL_AMOUNT <=',Monto_Hasta2,'] AND [#RISK >',Riesgo_Hasta2,']){@VERDADERO;}\nIF([#OP_DOL_AMOUNT >',Monto_Hasta2,'] AND [#RISK >',Riesgo_Hasta3,']){@VERDADERO;}\n@FALSO;}')

# OTRAS METRICAS

In [ ]:
# VER % DE CBKS REAL Y CBK LOW SIN SER TRANSACCIONAL
f, (ax1,ax2,ax3) = plt.subplots(3, figsize=(20,20))
sns.heatmap(DistrCbkTot,annot=True, ax=ax1,cmap="Blues",robust=True) # Distribución cbk total(LOW + MIDHIGH y con trsansaccionales)
sns.heatmap(Acbk,annot=True, ax=ax2,cmap="Blues",robust=True) # % cbk por celda 
sns.heatmap(AcbkT,annot=True, ax=ax3,cmap="Blues",robust=True) # % cbk Sin transaccional y sin Midhigh por celda

plt.show()

In [ ]:
# VER POR CANTIDADES
f, ((ax1, ax2),(ax3,ax4)) = plt.subplots(2,2, figsize=(30,15))
sns.heatmap(pivot_2,annot=True, ax=ax1,cmap="PiYG") #QTY incoming PF
sns.heatmap(pivot_AQ,annot=True, ax=ax2,cmap="Blues") # Qty aprobado LOW
sns.heatmap(pivot_cbk0 ,annot=True, ax=ax3) #Qty cbks
sns.heatmap(pivot_cbk01,annot=True, ax=ax4) # qty cbk LOW

In [ ]:
# #EXPORTAR

# import os
# from pandas import ExcelWriter
# from openpyxl import load_workbook


# #Creo archivo
# ruta = "C:/Users/mgaona/Desktop/Analisis/base python/MatrizRiesgo"+Seller+desde+hasta+".xlsx" 

# #Ingreso las bases a hojas al archivo
# df01.to_excel(ruta, index=False)

# # Cuando exporte, agregar la query en el excel!!!!